This analysis takes what happened in X minutes later and fit a gradient to this,

this, combining with RA should provide good enough factors to make decisions

In [1]:
import pandas as pd

import numpy as np

import datetime as dt

from collections import Counter

%matplotlib inline

import matplotlib.pyplot as plt

from mpl_toolkits import mplot3d

# Loading Data

In [2]:
XRP_Increase = pd.read_csv('../XRP_5min_Increase_SmallAmount_5min.csv')
XRP_Decrease = pd.read_csv('../XRP_5min_Decrease_SmallAmount_5min.csv')

In [3]:
original_columns = ['datetime_real','date','close','high','low','open','quoteVolume','volume','weightedAverage']

In [43]:
XRP_Analysis_table = XRP_Increase[original_columns].copy()

In [44]:
XRP_Analysis_table.head()

,datetime_real,date,close,high,low,open,quoteVolume,volume,weightedAverage
0,2017-05-04 06:20:00,1493875200,0.069288,0.069960,0.069288,0.069960,163.538132,11.431388,0.069900
1,2017-05-04 06:25:00,1493875500,0.069100,0.069949,0.069000,0.069949,113714.544855,7909.709770,0.069558
2,2017-05-04 06:30:00,1493875800,0.068000,0.069100,0.067500,0.069000,220670.240024,15126.256548,0.068547
3,2017-05-04 06:35:00,1493876100,0.068465,0.068465,0.066432,0.067800,271144.858354,18298.825581,0.067487
4,2017-05-04 06:40:00,1493876400,0.068646,0.068646,0.067517,0.067802,595249.214904,40678.358104,0.068338


In [45]:
section_period = {'20min': 4,
'1H': 12,
'2H': 24,
'4H': 48,
'12H': 144,
'24H': 288,
'5D': 1440,
'10D': 2880}

In [46]:
for k,v in section_period.items():
    print(k,v)

20min 4
1H 12
2H 24
4H 48
12H 144
24H 288
5D 1440
10D 2880


In [47]:
for k,v in section_period.items():
    print(k,v)
    XRP_Analysis_table['look_back_{}'.format(k)] = XRP_Analysis_table['close'].shift(v)   # i.e. 20min ago, this is what the close price is
    XRP_Analysis_table['look_forward_{}'.format(k)] = XRP_Analysis_table['close'].shift(-v)  # i.e. 20min later, the close price is


for k,v in section_period.items():
    print(k,v)
    XRP_Analysis_table['calculate_look_back_Percent_{}_in_minutes'.format(k)]    = 100*(XRP_Analysis_table['close'] - XRP_Analysis_table['look_back_{}'.format(k)])/(5*v)
    XRP_Analysis_table['calculate_look_forward_Percent_{}_in_minutes'.format(k)] =  100*(XRP_Analysis_table['look_forward_{}'.format(k)] - XRP_Analysis_table['close'])/(5*v)

20min 4
1H 12
2H 24
4H 48
12H 144
24H 288
5D 1440
10D 2880
20min 4
1H 12
2H 24
4H 48
12H 144
24H 288
5D 1440
10D 2880


In [63]:
for l_back in [d for d in XRP_Analysis_table.columns if 'calculate_look_back' in d]:
    for l_forward in [d for d in XRP_Analysis_table.columns if 'calculate_look_forward' in d]:
#         print(l_back, l_forward)
        XRP_Analysis_table['B:{0},F:{1}'.format(l_back.split('_')[-3], l_forward.split('_')[-3])] = list(zip(round(XRP_Analysis_table[l_back],4), round(XRP_Analysis_table[l_forward],4)))

calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_20min_in_minutes
calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_1H_in_minutes
calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_2H_in_minutes
calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_4H_in_minutes
calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_12H_in_minutes
calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_24H_in_minutes
calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_5D_in_minutes
calculate_look_back_Percent_20min_in_minutes calculate_look_forward_Percent_10D_in_minutes
calculate_look_back_Percent_1H_in_minutes calculate_look_forward_Percent_20min_in_minutes
calculate_look_back_Percent_1H_in_minutes calculate_look_forward_Percent_1H_in_minutes
calculate_look_back_Percent_1H_in_minutes calculate_look_forward_Percent_2H_in_minutes
calculate_

In [72]:
np.arange(-2, 2.4, 0.4)

array([-2.0000000e+00, -1.6000000e+00, -1.2000000e+00, -8.0000000e-01,
       -4.0000000e-01, -4.4408921e-16,  4.0000000e-01,  8.0000000e-01,
        1.2000000e+00,  1.6000000e+00,  2.0000000e+00])

In [73]:
bins = np.arange(-2, 2.4, 0.4)

In [80]:
bucketing = []

for l_forward in [d for d in XRP_Analysis_table.columns if 'calculate_look_forward' in d]:
    print(l_forward , ' : ', XRP_Analysis_table[l_forward].max(),  XRP_Analysis_table[l_forward].min())
    temp_data = XRP_Analysis_table[l_forward]
    bucketing.append([l_forward, np.histogram(temp_data, bins)])


calculate_look_forward_Percent_20min_in_minutes  :  1.7198200000000008 -1.685000000000001
calculate_look_forward_Percent_1H_in_minutes  :  0.9026666666666672 -0.8750000166666667
calculate_look_forward_Percent_2H_in_minutes  :  0.43833330833333334 -0.4402750416666668
calculate_look_forward_Percent_4H_in_minutes  :  0.23347619999999997 -0.26779285833333333
calculate_look_forward_Percent_12H_in_minutes  :  0.10652778611111113 -0.09922204166666665
calculate_look_forward_Percent_24H_in_minutes  :  0.07465277499999999 -0.058340418750000005
calculate_look_forward_Percent_5D_in_minutes  :  0.02083333402777778 -0.01887500013888889
calculate_look_forward_Percent_10D_in_minutes  :  0.01594979159722222 -0.012989872222222225


In [82]:
len(bucketing)

8

In [90]:
bucketing[0][1][0]

array([     1,      2,     13,    206, 114079, 121551,    218,     15,
            5,      2], dtype=int64)

In [127]:
for i in bucketing:
    print(i[0])
    temp_df = pd.DataFrame(i[1]).T
    temp_df[1] = round(temp_df[1],4)
    display(temp_df)

calculate_look_forward_Percent_20min_in_minutes


,0,1
0,1.0,-2.0
1,2.0,-1.6
2,13.0,-1.2
3,206.0,-0.8
4,114079.0,-0.4
5,121551.0,-0.0
6,218.0,0.4
7,15.0,0.8
8,5.0,1.2
9,2.0,1.6


calculate_look_forward_Percent_1H_in_minutes


,0,1
0,0.0,-2.0
1,0.0,-1.6
2,2.0,-1.2
3,19.0,-0.8
4,118720.0,-0.4
5,117303.0,-0.0
6,39.0,0.4
7,1.0,0.8
8,0.0,1.2
9,0.0,1.6


calculate_look_forward_Percent_2H_in_minutes


...

,0,1
0,0.0,-2.0
1,0.0,-1.6
2,0.0,-1.2
3,0.0,-0.8
4,123140.0,-0.4
5,112668.0,-0.0
6,0.0,0.4
7,0.0,0.8
8,0.0,1.2
9,0.0,1.6


calculate_look_forward_Percent_5D_in_minutes


,0,1
0,0.0,-2.0
1,0.0,-1.6
2,0.0,-1.2
3,0.0,-0.8
4,132979.0,-0.4
5,101677.0,-0.0
6,0.0,0.4
7,0.0,0.8
8,0.0,1.2
9,0.0,1.6


calculate_look_forward_Percent_10D_in_minutes


,0,1
0,0.0,-2.0
1,0.0,-1.6
2,0.0,-1.2
3,0.0,-0.8
4,134763.0,-0.4
5,98453.0,-0.0
6,0.0,0.4
7,0.0,0.8
8,0.0,1.2
9,0.0,1.6


In [110]:
bucketing[0][1][1][7]

0.7999999999999994

In [111]:
a = pd.DataFrame(bucketing[0][1]).T#.plot.bar()

In [113]:
a[1] = round(a[1],4)